In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics


# Data Source #1: REBEL + Ablations

In [31]:
data_df = pd.read_pickle('./data/rebel/data_post_entailment.pkl')

In [26]:
data_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,clean_claim_triples,main_text_triples,clean_main_text_triples,cleaned_claim_evidence_1,claim_evidence_1,cleaned_entailment_scores_1,entailment_scores_1,verdict,verdict_actual,clean_verdict
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,...,"[(A baby died at an unnamed medical facility, ...","[(Confederate flag, instance of, racist), (Fie...","[(Confederate flag, instance of, racist), (Fie...","{'(A baby died at an unnamed medical facility,...","{'(A baby died at an unnamed medical facility,...","[[0.66573095, -0.5662217, -0.4859993], [0.4305...","[[0.66573095, -0.56622165, -0.48599926], [0.63...",-1,0,-1
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...",...,"[(Shawnee County, located in the administrativ...","[(KSNT, located in the administrative territor...","[(KSNT, located in the administrative territor...","{'(Shawnee County, located in the administrati...","{'(Bat from Shawnee County, has cause, rabies)...","[[0.6284045, -0.6450901, -0.4347028], [0.61757...","[[0.62699527, -0.7306919, -0.27012262], [0.628...",-1,1,-1
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...",...,"[(banned pork from school canteens, country, G...","[(Express, country, British), (Express, instan...","[(Express, country, British), (Express, instan...","{'(banned pork from school canteens, country, ...","{'(banned pork from school canteens, country, ...","[[0.6420961, -0.6923114, -0.3292681], [0.68023...","[[0.6420961, -0.6923114, -0.3292681], [0.68787...",-1,-1,-1
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...",...,"[(Coronavirus, country, Canada), (safe drugs, ...","[(heroin, instance of, controlled substances),...","[(heroin, instance of, controlled substance), ...","{'(Coronavirus, country, Canada)': ['(coronavi...","{'(Coronavirus, country, Canada)': ['(coronavi...","[[-0.15032902, -0.75801086, 0.6346816], [0.660...","[[-0.10043568, -0.7755534, 0.6232412], [0.6620...",-1,1,-1
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...",...,"[(Wayne National Forest, instance of, wildlife...","[(oak forests, located in the administrative t...","[(oak forests, located in the administrative t...","{'(Wayne National Forest, instance of, wildlif...","{'(Wayne National Forest, instance of, forest)...","[[0.63135314, -0.7029956, -0.32739952], [0.662...","[[0.63064003, -0.6786143, -0.3765312], [0.6313...",-1,1,-1


In [35]:
data_df['verdict_actual'] = data_df['label'].apply(lambda label: 1 if label=='true' else (-1 if label=='false' else 0))

In [29]:
def get_verdict(row, low, high, cleaned=True, random=False):
    if random:
        entailment_scores = row['random_entailment_scores_1']
    else:
        if cleaned:
            entailment_scores = row['cleaned_entailment_scores_1']
        else:
            entailment_scores = row['entailment_scores_1']

    index_values = [-1, 1, 0]
    verdict = 0

    for subclaim_entailment_scores in entailment_scores:
        which_verdict = subclaim_entailment_scores.argmax()
        verdict += index_values[which_verdict]*subclaim_entailment_scores[which_verdict]

    if verdict > high:
        return 1
    elif verdict > low:
        return 0
    else:
        return -1

In [42]:
highs = np.arange(-1, 1, 0.01)
lows = np.arange(-1, 1, 0.01)

best_f1_score = -1
best_high = 0.1
best_low = -0.1
for high in highs:
    for low in lows:
        if low < high:
            data_df['verdict'] = data_df.apply(lambda row: get_verdict(row, low, high), axis=1)

            f1 = metrics.f1_score(data_df['verdict_actual'], data_df['verdict'], labels=[-1, 1, 0], average='weighted')

            if f1 > best_f1_score:
                best_f1_score = f1
                best_low = low
                best_high = high

print(best_f1_score)
print(best_low)
print(best_high)



0.36532800124454373
-0.6599999999999997
-0.6299999999999997


In [33]:
data_df['clean_verdict'] = data_df.apply(lambda row: get_verdict(row, best_low, best_high, cleaned=True), axis=1)
data_df['verdict'] = data_df.apply(lambda row: get_verdict(row, best_low, best_high, cleaned=False), axis=1)
data_df['random_verdict'] = data_df.apply(lambda row: get_verdict(row, best_low, best_high, cleaned=True, random=True), axis=1)


In [39]:
target_names = ['contradiction', 'support', 'nei']
labels = [-1, 1, 0]

print('With Redundancy Removal')
print(metrics.classification_report(data_df['verdict_actual'], data_df['clean_verdict'], target_names=target_names, labels=labels))
print("Accuracy Score: ", metrics.accuracy_score(data_df['verdict_actual'], data_df['clean_verdict']))

print()

print('Without Redundancy Removal')
print(metrics.classification_report(data_df['verdict_actual'], data_df['verdict'], target_names=target_names, labels=labels))
print("Accuracy Score: ", metrics.accuracy_score(data_df['verdict_actual'], data_df['verdict']))

print()

print('Entailment Randomized')
print(metrics.classification_report(data_df['verdict_actual'], data_df['random_verdict'], target_names=target_names, labels=labels))
print("Accuracy Score: ", metrics.accuracy_score(data_df['verdict_actual'], data_df['random_verdict']))


With Redundancy Removal
               precision    recall  f1-score   support

contradiction       0.28      0.27      0.27       768
      support       0.47      0.59      0.52      1228
          nei       0.20      0.07      0.11       456

     accuracy                           0.39      2452
    macro avg       0.31      0.31      0.30      2452
 weighted avg       0.36      0.39      0.37      2452

Accuracy Score:  0.39110929853181076

Without Redundancy Removal
               precision    recall  f1-score   support

contradiction       0.27      0.32      0.29       768
      support       0.45      0.52      0.48      1228
          nei       0.18      0.04      0.07       456

     accuracy                           0.37      2452
    macro avg       0.30      0.29      0.28      2452
 weighted avg       0.34      0.37      0.34      2452

Accuracy Score:  0.3674551386623165

Entailment Randomized
               precision    recall  f1-score   support

contradiction       

In [40]:
data_df.to_pickle('./data/data_verdict.pkl')

In [41]:
data_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,cleaned_claim_evidence_1,claim_evidence_1,cleaned_entailment_scores_1,entailment_scores_1,random_claim_evidence_1,random_entailment_scores_1,random_verdict,clean_verdict,verdict,verdict_actual
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,...,"{'(A baby died at an unnamed medical facility,...","{'(A baby died at an unnamed medical facility,...","[[0.66573095, -0.5662217, -0.4859993], [0.4305...","[[0.66573095, -0.56622165, -0.48599926], [0.63...","{'(A baby died at an unnamed medical facility,...","[[0.6385632, -0.5613695, -0.52640414], [0.6686...",-1,-1,-1,0
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...",...,"{'(Shawnee County, located in the administrati...","{'(Bat from Shawnee County, has cause, rabies)...","[[0.6284045, -0.6450901, -0.4347028], [0.61757...","[[0.62699527, -0.7306919, -0.27012262], [0.628...","{'(Shawnee County, located in the administrati...","[[0.6175279, -0.7271561, -0.2998387], [-0.5644...",1,-1,-1,1
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...",...,"{'(banned pork from school canteens, country, ...","{'(banned pork from school canteens, country, ...","[[0.6420961, -0.6923114, -0.3292681], [0.68023...","[[0.6420961, -0.6923114, -0.3292681], [0.68787...","{'(banned pork from school canteens, country, ...","[[0.679509, -0.6404532, -0.35789278], [0.69187...",-1,-1,-1,-1
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...",...,"{'(Coronavirus, country, Canada)': ['(coronavi...","{'(Coronavirus, country, Canada)': ['(coronavi...","[[-0.15032902, -0.75801086, 0.6346816], [0.660...","[[-0.10043568, -0.7755534, 0.6232412], [0.6620...","{'(Coronavirus, country, Canada)': ['(opioid, ...","[[0.69448966, -0.6130783, -0.37658876], [0.686...",-1,-1,-1,1
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...",...,"{'(Wayne National Forest, instance of, wildlif...","{'(Wayne National Forest, instance of, forest)...","[[0.63135314, -0.7029956, -0.32739952], [0.662...","[[0.63064003, -0.6786143, -0.3765312], [0.6313...","{'(Wayne National Forest, instance of, wildlif...","[[0.6313532, -0.7029955, -0.3273997], [-0.2482...",0,-1,-1,1


# Data Source #2: FRED

In [72]:
data_df = pd.read_pickle('./data/fred/data_post_entailment.pkl')

In [73]:
data_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,main_text_kg,FRED_claim_kg,FRED_main_text_kg,FRED_claim_kg_size,FRED_main_text_kg_size,FRED_claim_triples,FRED_main_text_triples,FRED_claim_evidence_1,FRED_entailment_scores_1
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,"[{'head': 'Confederate flag', 'type': 'instanc...",[{'head': 'A baby died at an unnamed medical f...,"[{'head': 'Confederate flag', 'type': 'instanc...",3,14,"[(A baby died at an unnamed medical facility, ...","[(Confederate flag, instance of, racist), (Fie...","{'(A baby died at an unnamed medical facility,...","[[0.66573095, -0.5662217, -0.4859993], [0.4305..."
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...","[{'head': 'KSNT', 'type': 'located in the admi...","[{'head': 'Bat from Shawnee County', 'type': '...","[{'head': 'report', 'type': 'Agent', 'tail': '...",3,131,"[(Shawnee County, located in the administrativ...","[(report, Agent, ksnt), (report, Theme, find),...","{'(Shawnee County, located in the administrati...","[[-0.6144093, 0.7461373, -0.25647667], [-0.518..."
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...","[{'head': 'Express', 'type': 'country', 'tail'...","[{'head': 'banned pork from school canteens', ...","[{'head': 'Express', 'type': 'country', 'tail'...",3,36,"[(banned pork from school canteens, country, G...","[(Express, country, British), (Express, instan...","{'(banned pork from school canteens, country, ...","[[0.6420961, -0.6923114, -0.3292681], [0.68023..."
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...","[{'head': 'heroin', 'type': 'instance of', 'ta...","[{'head': 'Coronavirus', 'type': 'country', 't...","[{'head': 'heroin', 'type': 'instance of', 'ta...",3,21,"[(Coronavirus, country, Canada), (safe drugs, ...","[(heroin, instance of, controlled substance), ...","{'(Coronavirus, country, Canada)': ['(coronavi...","[[-0.15032902, -0.75801086, 0.6346816], [0.660..."
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...","[{'head': 'oak forests', 'type': 'located in t...","[{'head': 'Wayne National Forest', 'type': 'in...","[{'head': 'oak forests', 'type': 'located in t...",3,3,"[(Wayne National Forest, instance of, wildlife...","[(oak forests, located in the administrative t...","{'(Wayne National Forest, instance of, wildlif...","[[0.63135314, -0.7029956, -0.32739952], [0.662..."


In [74]:
data_df['verdict_actual'] = data_df['label'].apply(lambda label: 1 if label=='true' else (-1 if label=='false' else 0))

In [79]:
def get_verdict(row, low, high):
    entailment_scores = row['FRED_entailment_scores_1']

    index_values = [-1, 1, 0]
    verdict = 0

    if entailment_scores is None:
        return 0

    for subclaim_entailment_scores in entailment_scores:
        which_verdict = subclaim_entailment_scores.argmax()
        verdict += index_values[which_verdict]*subclaim_entailment_scores[which_verdict]

    if verdict > high:
        return 1
    elif verdict > low:
        return 0
    else:
        return -1

In [80]:
highs = np.arange(-1, 1, 0.01)
lows = np.arange(-1, 1, 0.01)

best_f1_score = -1
best_high = 0.1
best_low = -0.1
for high in highs:
    for low in lows:
        if low < high:
            data_df['verdict'] = data_df.apply(lambda row: get_verdict(row, low, high), axis=1)

            f1 = metrics.f1_score(data_df['verdict_actual'], data_df['verdict'], labels=[-1, 1, 0], average='weighted')

            if f1 > best_f1_score:
                best_f1_score = f1
                best_low = low
                best_high = high

print(best_f1_score)
print(best_low)
print(best_high)



0.3637707961350442
-0.97
-0.94


In [82]:
data_df['verdict'] = data_df.apply(lambda row: get_verdict(row, low=-0.97, high=-0.94), axis=1)


In [83]:
target_names = ['contradiction', 'support', 'nei']
labels = [-1, 1, 0]

print(metrics.classification_report(data_df['verdict_actual'], data_df['verdict'], target_names=target_names, labels=labels))
print("Accuracy Score: ", metrics.accuracy_score(data_df['verdict_actual'], data_df['verdict']))


               precision    recall  f1-score   support

contradiction       0.30      0.38      0.33       768
      support       0.47      0.57      0.52      1228
          nei       0.14      0.00      0.00       455

     accuracy                           0.40      2451
    macro avg       0.30      0.32      0.28      2451
 weighted avg       0.36      0.40      0.36      2451

Accuracy Score:  0.40269277845777235


In [84]:
data_df.to_pickle('./data/fred/data_verdict.pkl')

In [85]:
data_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,FRED_claim_kg,FRED_main_text_kg,FRED_claim_kg_size,FRED_main_text_kg_size,FRED_claim_triples,FRED_main_text_triples,FRED_claim_evidence_1,FRED_entailment_scores_1,verdict_actual,verdict
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,...,[{'head': 'A baby died at an unnamed medical f...,"[{'head': 'Confederate flag', 'type': 'instanc...",3,14,"[(A baby died at an unnamed medical facility, ...","[(Confederate flag, instance of, racist), (Fie...","{'(A baby died at an unnamed medical facility,...","[[0.66573095, -0.5662217, -0.4859993], [0.4305...",0,-1
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...",...,"[{'head': 'Bat from Shawnee County', 'type': '...","[{'head': 'report', 'type': 'Agent', 'tail': '...",3,131,"[(Shawnee County, located in the administrativ...","[(report, Agent, ksnt), (report, Theme, find),...","{'(Shawnee County, located in the administrati...","[[-0.6144093, 0.7461373, -0.25647667], [-0.518...",1,1
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...",...,"[{'head': 'banned pork from school canteens', ...","[{'head': 'Express', 'type': 'country', 'tail'...",3,36,"[(banned pork from school canteens, country, G...","[(Express, country, British), (Express, instan...","{'(banned pork from school canteens, country, ...","[[0.6420961, -0.6923114, -0.3292681], [0.68023...",-1,-1
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...",...,"[{'head': 'Coronavirus', 'type': 'country', 't...","[{'head': 'heroin', 'type': 'instance of', 'ta...",3,21,"[(Coronavirus, country, Canada), (safe drugs, ...","[(heroin, instance of, controlled substance), ...","{'(Coronavirus, country, Canada)': ['(coronavi...","[[-0.15032902, -0.75801086, 0.6346816], [0.660...",1,1
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...",...,"[{'head': 'Wayne National Forest', 'type': 'in...","[{'head': 'oak forests', 'type': 'located in t...",3,3,"[(Wayne National Forest, instance of, wildlife...","[(oak forests, located in the administrative t...","{'(Wayne National Forest, instance of, wildlif...","[[0.63135314, -0.7029956, -0.32739952], [0.662...",1,-1


# Data Source #3: SpaCy

In [63]:
data_df = pd.read_pickle('./data/spacy/data_post_entailment.pkl')

In [64]:
data_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,main_text_triples,clean_main_text_triples,spacy_main_text_kg,spacy_claim_kg,spacy_claim_kg_size,spacy_main_text_kg_size,spacy_claim_triples,spacy_main_text_triples,spacy_claim_evidence_1,spacy_entailment_scores_1
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,...,"[(Confederate flag, instance of, racist), (Fie...","[(Confederate flag, instance of, racist), (Fie...","[{'head': '@FierceFemtivist', 'type': 'publish...","[{'head': 'baby', 'type': 'died', 'tail': 'fac...",2,54,"[(baby, died, facility), (nurse, died, child)]","[(@FierceFemtivist, published, series), (child...","{'(baby, died, facility)': ['(baby, reads, eve...","[[0.67679816, -0.60078514, -0.4254427], [0.677..."
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...",...,"[(KSNT, located in the administrative territor...","[(KSNT, located in the administrative territor...","[{'head': 'bat', 'type': 'reports', 'tail': 'C...","[{'head': 'Bat', 'type': 'tests', 'tail': 'Cou...",2,12,"[(Bat, tests, County), (Bat, tests, rabies)]","[(bat, reports, County), (Department, urging, ...","{'(Bat, tests, County)': ['(bat, reports, Coun...","[[-0.20893203, -0.7196208, 0.66218823], [0.569..."
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...",...,"[(Express, country, British), (Express, instan...","[(Express, country, British), (Express, instan...","[{'head': 'cafes', 'type': 'reported', 'tail':...","[{'head': 'Germany', 'type': 'banned', 'tail':...",3,168,"[(Germany, banned, pork), (Germany, banned, ca...","[(cafes, reported, products), (GERMANY, made, ...","{'(Germany, banned, pork)': ['(frenzy, banned,...","[[-0.20289786, -0.7068694, 0.67761946], [0.638..."
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...",...,"[(heroin, instance of, controlled substances),...","[(heroin, instance of, controlled substance), ...","[{'head': 'government', 'type': 'urged', 'tail...","[{'head': 'Coronavirus', 'type': 'prompts', 't...",3,114,"[(Coronavirus, prompts, drugs), (Coronavirus, ...","[(government, urged, provinces), (government, ...","{'(Coronavirus, prompts, drugs)': ['(who, ramp...","[[0.298262, -0.8961559, 0.3285489], [0.4528000..."
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...",...,"[(oak forests, located in the administrative t...","[(oak forests, located in the administrative t...","[{'head': 'who', 'type': 'say', 'tail': 'plant...","[{'head': 'Forest', 'type': 'plans', 'tail': '...",2,16,"[(Forest, plans, fires), (Forest, plans, tree)]","[(who, say, plants), (fire, say, forests), (bl...","{'(Forest, plans, fires)': ['(fire, say, fores...","[[-0.70220375, 0.71068317, -0.0428862], [-0.46..."


In [65]:
data_df['verdict_actual'] = data_df['label'].apply(lambda label: 1 if label=='true' else (-1 if label=='false' else 0))

In [66]:
def get_verdict(row, low, high):
    entailment_scores = row['spacy_entailment_scores_1']

    index_values = [-1, 1, 0]
    verdict = 0

    if entailment_scores is None:
        return 0

    for subclaim_entailment_scores in entailment_scores:
        which_verdict = subclaim_entailment_scores.argmax()
        verdict += index_values[which_verdict]*subclaim_entailment_scores[which_verdict]

    if verdict > high:
        return 1
    elif verdict > low:
        return 0
    else:
        return -1

In [67]:
highs = np.arange(-1, 1, 0.01)
lows = np.arange(-1, 1, 0.01)

best_f1_score = -1
best_high = 0.1
best_low = -0.1
for high in highs:
    for low in lows:
        if low < high:
            data_df['verdict'] = data_df.apply(lambda row: get_verdict(row, low, high), axis=1)

            f1 = metrics.f1_score(data_df['verdict_actual'], data_df['verdict'], labels=[-1, 1, 0], average='weighted')

            if f1 > best_f1_score:
                best_f1_score = f1
                best_low = low
                best_high = high

print(best_f1_score)
print(best_low)
print(best_high)



0.38956532618465206
-0.96
-0.7199999999999998


In [68]:
data_df['verdict'] = data_df.apply(lambda row: get_verdict(row, low=-0.96, high=-0.7199999999999998), axis=1)


In [69]:
target_names = ['contradiction', 'support', 'nei']
labels = [-1, 1, 0]

print(metrics.classification_report(data_df['verdict_actual'], data_df['verdict'], target_names=target_names, labels=labels))
print("Accuracy Score: ", metrics.accuracy_score(data_df['verdict_actual'], data_df['verdict']))


               precision    recall  f1-score   support

contradiction       0.35      0.27      0.31       768
      support       0.49      0.58      0.53      1228
          nei       0.16      0.14      0.15       456

     accuracy                           0.40      2452
    macro avg       0.33      0.33      0.33      2452
 weighted avg       0.38      0.40      0.39      2452

Accuracy Score:  0.40212071778140296


In [70]:
data_df.to_pickle('./data/spacy/data_verdict.pkl')

In [71]:
data_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,spacy_main_text_kg,spacy_claim_kg,spacy_claim_kg_size,spacy_main_text_kg_size,spacy_claim_triples,spacy_main_text_triples,spacy_claim_evidence_1,spacy_entailment_scores_1,verdict_actual,verdict
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,...,"[{'head': '@FierceFemtivist', 'type': 'publish...","[{'head': 'baby', 'type': 'died', 'tail': 'fac...",2,54,"[(baby, died, facility), (nurse, died, child)]","[(@FierceFemtivist, published, series), (child...","{'(baby, died, facility)': ['(baby, reads, eve...","[[0.67679816, -0.60078514, -0.4254427], [0.677...",0,-1
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...",...,"[{'head': 'bat', 'type': 'reports', 'tail': 'C...","[{'head': 'Bat', 'type': 'tests', 'tail': 'Cou...",2,12,"[(Bat, tests, County), (Bat, tests, rabies)]","[(bat, reports, County), (Department, urging, ...","{'(Bat, tests, County)': ['(bat, reports, Coun...","[[-0.20893203, -0.7196208, 0.66218823], [0.569...",1,1
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...",...,"[{'head': 'cafes', 'type': 'reported', 'tail':...","[{'head': 'Germany', 'type': 'banned', 'tail':...",3,168,"[(Germany, banned, pork), (Germany, banned, ca...","[(cafes, reported, products), (GERMANY, made, ...","{'(Germany, banned, pork)': ['(frenzy, banned,...","[[-0.20289786, -0.7068694, 0.67761946], [0.638...",-1,-1
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...",...,"[{'head': 'government', 'type': 'urged', 'tail...","[{'head': 'Coronavirus', 'type': 'prompts', 't...",3,114,"[(Coronavirus, prompts, drugs), (Coronavirus, ...","[(government, urged, provinces), (government, ...","{'(Coronavirus, prompts, drugs)': ['(who, ramp...","[[0.298262, -0.8961559, 0.3285489], [0.4528000...",1,1
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...",...,"[{'head': 'who', 'type': 'say', 'tail': 'plant...","[{'head': 'Forest', 'type': 'plans', 'tail': '...",2,16,"[(Forest, plans, fires), (Forest, plans, tree)]","[(who, say, plants), (fire, say, forests), (bl...","{'(Forest, plans, fires)': ['(fire, say, fores...","[[-0.70220375, 0.71068317, -0.0428862], [-0.46...",1,1


# Data Source #4: LLaMa

In [54]:
data_df = pd.read_pickle('./data/llama/data_post_entailment.pkl')

In [55]:
data_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,main_text_triples,clean_main_text_triples,llm_claim_kg,llm_main_text_kg,llama_claim_kg_size,llama_main_text_kg_size,llama_claim_triples,llama_main_text_triples,llama_claim_evidence_1,llama_entailment_scores_1
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,...,"[(Confederate flag, instance of, racist), (Fie...","[(Confederate flag, instance of, racist), (Fie...","[{'head': 'baby', 'type': 'died', 'tail': 'med...","[{'head': 'Confederate flag', 'type': 'instanc...",2,14,"[(baby, died, medical-facility), (medical-faci...","[(Confederate flag, instance of, racist), (Fie...","{'(baby, died, medical-facility)': ['(cardiac ...","[[-0.08809295, -0.7930231, 0.60278845], [0.649..."
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...",...,"[(KSNT, located in the administrative territor...","[(KSNT, located in the administrative territor...","[{'head': 'Bat', 'type': 'from', 'tail': 'Shaw...","[{'head': 'KSNT', 'type': 'located in the admi...",2,6,"[(Bat, from, Shawnee County), (Shawnee County,...","[(KSNT, located in the administrative territor...","{'(Bat, from, Shawnee County)': ['(Topeka, loc...","[[0.6422565, -0.5995739, -0.477512], [-0.28233..."
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...",...,"[(Express, country, British), (Express, instan...","[(Express, country, British), (Express, instan...","[{'head': 'Germany', 'type': 'has', 'tail': 'b...","[{'head': 'Express', 'type': 'country', 'tail'...",2,36,"[(Germany, has, banned), (pork, from, school c...","[(Express, country, British), (Express, instan...","{'(Germany, has, banned)': ['(Germany, contine...","[[0.51010597, -0.85914826, 0.04069696], [-0.26..."
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...",...,"[(heroin, instance of, controlled substances),...","[(heroin, instance of, controlled substance), ...","[{'head': 'Coronavirus', 'type': 'prompts', 't...","[{'head': 'heroin', 'type': 'instance of', 'ta...",2,21,"[(Coronavirus, prompts, Canada), (Canada, roll...","[(heroin, instance of, controlled substance), ...","{'(Coronavirus, prompts, Canada)': ['(coronavi...","[[-0.7723689, 0.030968107, 0.6344187], [-0.440..."
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...",...,"[(oak forests, located in the administrative t...","[(oak forests, located in the administrative t...","[{'head': 'Wayne National Forest', 'type': 'pl...","[{'head': 'oak forests', 'type': 'located in t...",2,3,"[(Wayne National Forest, plans, fires), (tree,...","[(oak forests, located in the administrative t...","{'(Wayne National Forest, plans, fires)': ['(o...","[[0.6409391, -0.7056944, -0.30198097], [-0.570..."


In [56]:
data_df['verdict_actual'] = data_df['label'].apply(lambda label: 1 if label=='true' else (-1 if label=='false' else 0))

In [57]:
def get_verdict(row, low, high):
    entailment_scores = row['llama_entailment_scores_1']

    index_values = [-1, 1, 0]
    verdict = 0

    for subclaim_entailment_scores in entailment_scores:
        which_verdict = subclaim_entailment_scores.argmax()
        verdict += index_values[which_verdict]*subclaim_entailment_scores[which_verdict]

    if verdict > high:
        return 1
    elif verdict > low:
        return 0
    else:
        return -1

In [58]:
highs = np.arange(-1, 1, 0.01)
lows = np.arange(-1, 1, 0.01)

best_f1_score = -1
best_high = 0.1
best_low = -0.1
for high in highs:
    for low in lows:
        if low < high:
            data_df['verdict'] = data_df.apply(lambda row: get_verdict(row, low, high), axis=1)

            f1 = metrics.f1_score(data_df['verdict_actual'], data_df['verdict'], labels=[-1, 1, 0], average='weighted')

            if f1 > best_f1_score:
                best_f1_score = f1
                best_low = low
                best_high = high

print(best_f1_score)
print(best_low)
print(best_high)



0.3706535840511861
-1.0
-0.95


In [59]:
data_df['verdict'] = data_df.apply(lambda row: get_verdict(row, best_low, best_high), axis=1)


In [60]:
target_names = ['contradiction', 'support', 'nei']
labels = [-1, 1, 0]

print(metrics.classification_report(data_df['verdict_actual'], data_df['verdict'], target_names=target_names, labels=labels))
print("Accuracy Score: ", metrics.accuracy_score(data_df['verdict_actual'], data_df['verdict']))


               precision    recall  f1-score   support

contradiction       0.29      0.28      0.28       768
      support       0.48      0.66      0.56      1228
          nei       0.27      0.01      0.02       456

     accuracy                           0.42      2452
    macro avg       0.34      0.32      0.29      2452
 weighted avg       0.38      0.42      0.37      2452

Accuracy Score:  0.42088091353996737


In [61]:
data_df.to_pickle('./data/llama/data_verdict.pkl')

In [62]:
data_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,llm_claim_kg,llm_main_text_kg,llama_claim_kg_size,llama_main_text_kg_size,llama_claim_triples,llama_main_text_triples,llama_claim_evidence_1,llama_entailment_scores_1,verdict_actual,verdict
0,34656,A baby died at an unnamed medical facility be...,"November 10, 2015",Fellow Twitter users suggested @FierceFemtivis...,Kim LaCapria,"On 8 November 2015, former Twitter user @Fierc...",http://webcache.googleusercontent.com/search?q...,unproven,"Politics, fiercefemtivist, racism",[{'head': 'A baby died at an unnamed medical f...,...,"[{'head': 'baby', 'type': 'died', 'tail': 'med...","[{'head': 'Confederate flag', 'type': 'instanc...",2,14,"[(baby, died, medical-facility), (medical-faci...","[(Confederate flag, instance of, racist), (Fie...","{'(baby, died, medical-facility)': ['(cardiac ...","[[-0.08809295, -0.7930231, 0.60278845], [0.649...",0,1
1,3632,Bat from Shawnee County tests positive for rab...,NaN,A bat found in northeastern Kansas has tested ...,,Topeka television station KSNT reports that th...,https://www.ksnt.com/news/bat-tests-positive-f...,true,"Rabies, Health, General News, Kansas, Bats, To...","[{'head': 'Bat from Shawnee County', 'type': '...",...,"[{'head': 'Bat', 'type': 'from', 'tail': 'Shaw...","[{'head': 'KSNT', 'type': 'located in the admi...",2,6,"[(Bat, from, Shawnee County), (Shawnee County,...","[(KSNT, located in the administrative territor...","{'(Bat, from, Shawnee County)': ['(Topeka, loc...","[[0.6422565, -0.5995739, -0.477512], [-0.28233...",1,1
2,29558,Germany has banned pork from school canteens b...,"March 7, 2016",What's true: Some politicians complained that ...,Kim LaCapria,"On 7 March 2016, British tabloid Express repor...",http://bnp.org.uk/news/regional/bnp-victory-br...,false,Politics,"[{'head': 'banned pork from school canteens', ...",...,"[{'head': 'Germany', 'type': 'has', 'tail': 'b...","[{'head': 'Express', 'type': 'country', 'tail'...",2,36,"[(Germany, has, banned), (pork, from, school c...","[(Express, country, British), (Express, instan...","{'(Germany, has, banned)': ['(Germany, contine...","[[0.51010597, -0.85914826, 0.04069696], [-0.26...",-1,1
3,8416,Coronavirus prompts Canada to roll out safe dr...,"April 16, 2020",Canada’s Pacific province of British Columbia ...,Tessa Vikander,"In March, the Canadian government urged provin...",,true,Health News,"[{'head': 'Coronavirus', 'type': 'country', 't...",...,"[{'head': 'Coronavirus', 'type': 'prompts', 't...","[{'head': 'heroin', 'type': 'instance of', 'ta...",2,21,"[(Coronavirus, prompts, Canada), (Canada, roll...","[(heroin, instance of, controlled substance), ...","{'(Coronavirus, prompts, Canada)': ['(coronavi...","[[-0.7723689, 0.030968107, 0.6344187], [-0.440...",1,1
4,7169,"Wayne National Forest plans fires for tree, wi...",NaN,"Nearly 2,000 acres of Wayne National Forest in...",,Forest officials say scientists who study nati...,,true,"Plants, Wildlife, Health, Wildlife health, For...","[{'head': 'Wayne National Forest', 'type': 'in...",...,"[{'head': 'Wayne National Forest', 'type': 'pl...","[{'head': 'oak forests', 'type': 'located in t...",2,3,"[(Wayne National Forest, plans, fires), (tree,...","[(oak forests, located in the administrative t...","{'(Wayne National Forest, plans, fires)': ['(o...","[[0.6409391, -0.7056944, -0.30198097], [-0.570...",1,1


# Data Source #5: ChatGPT (small sample)

In [86]:
data_df = pd.read_pickle('./data/chatgpt/data_post_entailment.pkl')

In [87]:
data_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,main_text_triples,clean_main_text_triples,chatgpt_claim_kg,chatgpt_main_text_kg,chatgpt_claim_kg_size,chatgpt_main_text_kg_size,chatgpt_claim_triples,chatgpt_main_text_triples,chatgpt_claim_evidence_1,chatgpt_entailment_scores_1
0,37068,Several emails circulating about the Obama ad...,"March 17, 2015",Various emails about the proposed Health Care ...,Rich Buhler & Staff,Many eRumors and opinions are still circulatin...,"http://www.amish.net/faq.asp, http://thomas.lo...",mixture,"Health / Medical, Medical, Politics",[{'head': 'proposed Universal Health Care bill...,...,"[(Patient Protection and Affordable Care Act, ...","[(Patient Protection and Affordable Care Act, ...","[{'head': 'Obama administration', 'type': 'pro...","[{'head': 'Health Care Reform Bill', 'type': '...",3,18,"[(Obama administration, proposed, Universal He...","[(Health Care Reform Bill, proposed, HR-3200),...","{'(Obama administration, proposed, Universal H...","[[0.4088891, -0.8990172, 0.1567729], [0.677720..."
1,9048,Proper exercise can reverse damage from heart ...,"January 9, 2018",Getty Images This news release describes a sma...,"Michael Joyce, MD,Adam Cifu, MD,Kevin Lomangino",The intervention proposed is at least a half-h...,http://circ.ahajournals.org/content/early/2018...,mixture,exercise,"[{'head': 'Proper exercise', 'type': 'has effe...",...,"[(left ventricle, part of, heart), (heart, has...","[(heart, has part, left ventricle), (left vent...","[{'head': 'exercise', 'type': 'canReverse', 't...","[{'head': 'study', 'type': 'documentsOutcome',...",1,16,"[(exercise, canReverse, damage from heart aging)]","[(study, documentsOutcome, 18% improvement in ...","{'(exercise, canReverse, damage from heart agi...","[[-0.4780542, -0.49286932, 0.72701037]]"
2,11375,Getting the focus back: There’s new hope for p...,"March 5, 2007",Age-related macular degeneration (AMD) is a si...,,The story does not mention the cost of Lucenti...,,mixture,,"[{'head': 'macular degeneration', 'type': 'sub...",...,"[(Lucentis, different from, Avastin), (Avastin...","[(Avastin, different from, Lucentis), (infecti...","[{'head': 'people', 'type': 'haveHopeFor', 'ta...","[{'head': 'story', 'type': 'doesNotMention', '...",1,21,"[(people, haveHopeFor, age-related macular deg...","[(story, doesNotMention, cost of Lucentis), (L...","{'(people, haveHopeFor, age-related macular de...","[[-0.45176712, -0.5446173, 0.7066105]]"
3,32571,"Jordan's King Abdullah II murdered his wife, Q...","June 27, 2016","— Rania Al Abdullah (@QueenRania) June 28, 2016",Brooke Binkowski,"On 24 June 2016, a shocking story suddenly app...",https://twitter.com/QueenRania/status/74768801...,false,"Junk News, awd news, jordan, king abdullah ii","[{'head': 'King Abdullah II', 'type': 'spouse'...",...,"[(Queen Rania, date of death, 24 June 2016), (...","[(Queen Rania, date of death, 24 June 2016), (...","[{'head': 'King Abdullah II', 'type': 'murdere...","[{'head': 'story', 'type': 'appearedOutofNowhe...",1,27,"[(King Abdullah II, murdered, Queen Rania)]","[(story, appearedOutofNowhere, On 24 June 2016...","{'(King Abdullah II, murdered, Queen Rania)': ...","[[0.6457535, -0.6481404, -0.40362918]]"
4,949,Biblical bad guys the ancient Philistines came...,"July 3, 2019","The ancient Philistines, the Biblical villains...","Rinat Harash, Ari Rabinovitch",The genetic findings came from skeletons unear...,https://reut.rs/2LBBkxd,true,Science News,"[{'head': 'Philistines', 'type': 'present in w...",...,"[(Delilah, spouse, Samson), (Samson, spouse, D...","[(Samson, spouse, Delilah), (Goliath, instance...","[{'head': 'Philistines', 'type': 'origin', 'ta...","[{'head': 'genetic findings', 'type': 'source'...",5,35,"[(Philistines, origin, Europe), (Philistines, ...","[(genetic findings, source, skeletons), (archa...","{'(Philistines, origin, Europe)': ['(Philistin...","[[0.61484134, -0.7440453, -0.26147], [0.630895..."


In [88]:
data_df['verdict_actual'] = data_df['label'].apply(lambda label: 1 if label=='true' else (-1 if label=='false' else 0))

In [89]:
def get_verdict(row, low, high):
    entailment_scores = row['chatgpt_entailment_scores_1']

    index_values = [-1, 1, 0]
    verdict = 0

    for subclaim_entailment_scores in entailment_scores:
        which_verdict = subclaim_entailment_scores.argmax()
        verdict += index_values[which_verdict]*subclaim_entailment_scores[which_verdict]

    if verdict > high:
        return 1
    elif verdict > low:
        return 0
    else:
        return -1

In [90]:
highs = np.arange(-1, 1, 0.01)
lows = np.arange(-1, 1, 0.01)

best_f1_score = -1
best_high = 0.1
best_low = -0.1
for high in highs:
    for low in lows:
        if low < high:
            data_df['verdict'] = data_df.apply(lambda row: get_verdict(row, low, high), axis=1)

            f1 = metrics.f1_score(data_df['verdict_actual'], data_df['verdict'], labels=[-1, 1, 0], average='weighted')

            if f1 > best_f1_score:
                best_f1_score = f1
                best_low = low
                best_high = high

print(best_f1_score)
print(best_low)
print(best_high)



0.42771929824561394
-0.6299999999999997
-0.49999999999999956


In [91]:
data_df['verdict'] = data_df.apply(lambda row: get_verdict(row, low=-0.6299999999999997, high=-0.49999999999999956), axis=1)


In [92]:
target_names = ['contradiction', 'support', 'nei']
labels = [-1, 1, 0]

print(metrics.classification_report(data_df['verdict_actual'], data_df['verdict'], target_names=target_names, labels=labels))
print("Accuracy Score: ", metrics.accuracy_score(data_df['verdict_actual'], data_df['verdict']))


               precision    recall  f1-score   support

contradiction       0.36      0.31      0.33        13
      support       0.52      0.71      0.60        24
          nei       0.33      0.15      0.21        13

     accuracy                           0.46        50
    macro avg       0.40      0.39      0.38        50
 weighted avg       0.43      0.46      0.43        50

Accuracy Score:  0.46


In [93]:
data_df.to_pickle('./data/chatgpt/data_verdict.pkl')

In [94]:
data_df.head()

,claim_id,claim,date_published,explanation,fact_checkers,main_text,sources,label,subjects,claim_kg,...,chatgpt_claim_kg,chatgpt_main_text_kg,chatgpt_claim_kg_size,chatgpt_main_text_kg_size,chatgpt_claim_triples,chatgpt_main_text_triples,chatgpt_claim_evidence_1,chatgpt_entailment_scores_1,verdict_actual,verdict
0,37068,Several emails circulating about the Obama ad...,"March 17, 2015",Various emails about the proposed Health Care ...,Rich Buhler & Staff,Many eRumors and opinions are still circulatin...,"http://www.amish.net/faq.asp, http://thomas.lo...",mixture,"Health / Medical, Medical, Politics",[{'head': 'proposed Universal Health Care bill...,...,"[{'head': 'Obama administration', 'type': 'pro...","[{'head': 'Health Care Reform Bill', 'type': '...",3,18,"[(Obama administration, proposed, Universal He...","[(Health Care Reform Bill, proposed, HR-3200),...","{'(Obama administration, proposed, Universal H...","[[0.4088891, -0.8990172, 0.1567729], [0.677720...",0,-1
1,9048,Proper exercise can reverse damage from heart ...,"January 9, 2018",Getty Images This news release describes a sma...,"Michael Joyce, MD,Adam Cifu, MD,Kevin Lomangino",The intervention proposed is at least a half-h...,http://circ.ahajournals.org/content/early/2018...,mixture,exercise,"[{'head': 'Proper exercise', 'type': 'has effe...",...,"[{'head': 'exercise', 'type': 'canReverse', 't...","[{'head': 'study', 'type': 'documentsOutcome',...",1,16,"[(exercise, canReverse, damage from heart aging)]","[(study, documentsOutcome, 18% improvement in ...","{'(exercise, canReverse, damage from heart agi...","[[-0.4780542, -0.49286932, 0.72701037]]",0,1
2,11375,Getting the focus back: There’s new hope for p...,"March 5, 2007",Age-related macular degeneration (AMD) is a si...,,The story does not mention the cost of Lucenti...,,mixture,,"[{'head': 'macular degeneration', 'type': 'sub...",...,"[{'head': 'people', 'type': 'haveHopeFor', 'ta...","[{'head': 'story', 'type': 'doesNotMention', '...",1,21,"[(people, haveHopeFor, age-related macular deg...","[(story, doesNotMention, cost of Lucentis), (L...","{'(people, haveHopeFor, age-related macular de...","[[-0.45176712, -0.5446173, 0.7066105]]",0,1
3,32571,"Jordan's King Abdullah II murdered his wife, Q...","June 27, 2016","— Rania Al Abdullah (@QueenRania) June 28, 2016",Brooke Binkowski,"On 24 June 2016, a shocking story suddenly app...",https://twitter.com/QueenRania/status/74768801...,false,"Junk News, awd news, jordan, king abdullah ii","[{'head': 'King Abdullah II', 'type': 'spouse'...",...,"[{'head': 'King Abdullah II', 'type': 'murdere...","[{'head': 'story', 'type': 'appearedOutofNowhe...",1,27,"[(King Abdullah II, murdered, Queen Rania)]","[(story, appearedOutofNowhere, On 24 June 2016...","{'(King Abdullah II, murdered, Queen Rania)': ...","[[0.6457535, -0.6481404, -0.40362918]]",-1,-1
4,949,Biblical bad guys the ancient Philistines came...,"July 3, 2019","The ancient Philistines, the Biblical villains...","Rinat Harash, Ari Rabinovitch",The genetic findings came from skeletons unear...,https://reut.rs/2LBBkxd,true,Science News,"[{'head': 'Philistines', 'type': 'present in w...",...,"[{'head': 'Philistines', 'type': 'origin', 'ta...","[{'head': 'genetic findings', 'type': 'source'...",5,35,"[(Philistines, origin, Europe), (Philistines, ...","[(genetic findings, source, skeletons), (archa...","{'(Philistines, origin, Europe)': ['(Philistin...","[[0.61484134, -0.7440453, -0.26147], [0.630895...",1,1
